# Orchestrate Multiple-Agents for High-Quality Research with AutoGen and LangChain Tools

This notebook showcases how to harness the combined power of **AutoGen** and **LangChain** tools to automate and elevate deep research workflows. At its core, the system coordinates a network of specialized agents—each executing a distinct role in the research and report generation workflow. Together, these agents collect data, analyze findings, and produce polished, insight-driven reports.


[Open in Colab](https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb) <a href="https://colab.research.google.com/github/miztiik/taars/blob/master/notebooks/deepresearch_w_autogen_langchain_tools.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 🧠 Agent Roles Overview

- **🧭 Planner Agent**  
  Defines the research scope, objectives, and key questions.  
  → Anchors the process by structuring it around a well-defined `__TASK_QUERY`.

- **🔍 Researcher Agent**  
  Gathers information using tools like `wiki_search` and `duckduckgo_search`.  
  → Supplies the system with relevant, factual data.

- **🧪 Critic Agent**  
  Reviews the research plan, gathered evidence, and drafted reports.  
  → Provides feedback to enforce analytical rigor and clarity.

- **✍️ Editor Agent**  
  Refines the final report's language, structure, and presentation.  
  → Ensures clarity, polish, and alignment with stakeholder expectations.


In [ ]:
%%capture --no-stderr

%pip install -qU autogen-agentchat
%pip install -qU autogen-ext

%pip install -qU loguru

%pip install -qU langchain
%pip install -qU langchain-community
%pip install -qU wikipedia
%pip install -qU lxml
%pip install -qU ddgs


In [ ]:
%load_ext autoreload
%autoreload 2
# %aimport -langchain_community
# Automatically reload modules before executing code

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html


In [2]:
from pathlib import Path
from loguru import logger
import sys

# import logging
# logger = logging.getLogger("")


# # Remove all existing handlers
# for handler in logger.handlers[:]:
#     logger.removeHandler(handler)

# REMOVE ALL EXISTING HANDLERS for LOGURU
if len(logger._core.handlers) > 1:
    logger.configure(handlers=[])

logger.add(
    sys.stderr,
    format="{time:HH:mm:ss:SSS} | {level} | {name}:{line} | {message}",
    level="INFO",
    colorize=True,
)

# Configure file logging with better settings

notebook_dir = Path.cwd()
log_file = notebook_dir / "logs" / "llm_execution.log"
log_file.parent.mkdir(exist_ok=True)
logger.add(
    log_file,
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level: <8} | {name}:{function}:{line} | {message}",
    level="DEBUG",
    rotation="10 MB",
    retention="7 days",
    compression="zip",
    enqueue=True,
)
logger.info(f"✅ Logging configured - File: {log_file}")


2025-08-19 14:51:51.893 | INFO     | __main__:<module>:38 - ✅ Logging configured - File: c:\Users\kumarsnaveen\Documents\NawiN\personal\gitrepos\taars\notebooks\logs\llm_execution.log
14:51:51:893 | INFO | __main__:38 | ✅ Logging configured - File: c:\Users\kumarsnaveen\Documents\NawiN\personal\gitrepos\taars\notebooks\logs\llm_execution.log


In [3]:
import os
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo


# Confirm the API key is set
assert os.environ["GEMINI_API_KEY"], "GEMINI_API_KEY is not set"
assert os.environ["GEMINI_MODEL_NAME"], "GEMINI_MODEL_NAME is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPM"], "RATE_LIMIT_GEMINI_RPM is not set"
assert os.environ["RATE_LIMIT_GEMINI_RPD"], "RATE_LIMIT_GEMINI_RPD is not set"
assert os.environ["GEMINI_BASE_URL"], "GEMINI_BASE_URL is not set"


model_info = ModelInfo(
    vision=True,
    function_calling=True,
    json_output=True,
    family=os.environ["GEMINI_MODEL_NAME"],
    structured_output=True,
)

model_client = OpenAIChatCompletionClient(
    model=os.environ["GEMINI_MODEL_NAME"],
    api_key=os.environ["GEMINI_API_KEY"],
    base_url=os.environ["GEMINI_BASE_URL"],
    model_info=model_info,
)


In [4]:
import asyncio, json, os
import tenacity
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.messages import StopMessage
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage


from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool
from ddgs import DDGS


In [5]:
## Agent Tools

from pathlib import Path
from datetime import datetime
import re
from typing import Optional, Dict, Any

# Wiki Search Tool
wiki_api = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api)


def wiki_search(q: str) -> dict:
    """Return structured output including text and source."""
    try:
        result = wiki_tool.run(q)
        return {"text": result, "source": "Wikipedia"}
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}

# DuckDuckGo Search Tool
def duckduckgo_search(q: str) -> dict:
    """Search the web using DuckDuckGo for information."""
    try:
        results = DDGS().text(q, region="us-en", safesearch="off", max_results=12)
        return {
            "text": results,
            "source": "DuckDuckGo",
            "query": q,
            "results_found": len(results),
        }
    except Exception as e:
        return {"text": f"Error: {str(e)}", "source": None}

def save_report(content: str, task_description: str, reports_dir: str = "reports") -> Dict[str, Any]:
    """
    Save a report to disk with automatic filename generation.
    
    This tool creates a timestamped Markdown file in the reports directory.
    Perfect for preserving research findings, analysis results, or final reports.
    
    Args:
        content (str): The report content to save. Can be plain text or Markdown.
                      If no title (# header) is present, one will be auto-generated.
        task_description (str): Brief description of the task/topic for filename.
                               Example: "AI impact on power sector analysis"
        reports_dir (str, optional): Directory to save reports. Defaults to "reports".
                                   Directory will be created if it doesn't exist.
    
    Returns:
        Dict[str, Any]: Result dictionary containing:
            - status: "success" or "error"
            - filepath: Full path to saved file (if successful)
            - filename: Just the filename (if successful)
            - error: Error message (if failed)
    
    Example Usage for Agents:
        # Save research findings
        result = save_report(
            content="# Market Analysis\\n\\nKey findings: ...",
            task_description="cryptocurrency market trends 2024"
        )
        
        # Save final report
        result = save_report(
            content=my_report_text,
            task_description="climate change impact assessment"
        )
        
        # Check if save was successful
        if result["status"] == "success":
            print(f"Report saved as: {result['filename']}")
        else:
            print(f"Save failed: {result['error']}")
    
    Generated Filename Format:
        YYYYMMDD_HHMM_meaningful_task_words.md
        Example: 20250819_1430_ai_power_sector.md
    
    Note: If file exists, automatic numbering prevents overwrites:
          20250819_1430_ai_power_sector_1.md, _2.md, etc.
    """
    try:
        # Ensure reports directory exists
        reports_path = Path(reports_dir)
        reports_path.mkdir(exist_ok=True)
        
        # Generate timestamped filename
        timestamp = datetime.now()
        date_time = timestamp.strftime("%Y%m%d_%H%M")
        task_name = _extract_task_name(task_description)
        
        filename = f"{date_time}_{task_name}.md"
        filepath = reports_path / filename
        
        # Handle filename conflicts with counter
        counter = 1
        while filepath.exists():
            filename = f"{date_time}_{task_name}_{counter}.md"
            filepath = reports_path / filename
            counter += 1
        
        # Format content with title if needed
        formatted_content = _format_content(content, task_description, timestamp)
        
        # Save to disk
        filepath.write_text(formatted_content, encoding='utf-8')
        logger.info(f"📄 Report saved: {filepath}")
        
        return {
            "status": "success",
            "filepath": str(filepath),
            "filename": filename,
            "timestamp": timestamp.isoformat()
        }
        
    except Exception as e:
        error_msg = f"Failed to save report: {str(e)}"
        logger.error(f"❌ {error_msg}")
        return {
            "status": "error",
            "error": error_msg,
            "task": task_description
        }


def _extract_task_name(task: str) -> str:
    """
    Extract 2-3 meaningful words from task description for filename.
    
    Args:
        task: The task description string
        
    Returns:
        Underscore-separated words suitable for filename
    """
    # Clean special characters and normalize
    clean_task = re.sub(r'[^\w\s]', ' ', task.lower())
    words = [w for w in clean_task.split() if len(w) > 2]
    
    # Filter common stop words
    stop_words = {
        'the', 'and', 'for', 'with', 'from', 'about', 'into', 'through',
        'during', 'before', 'after', 'above', 'below', 'over', 'under'
    }
    meaningful = [w for w in words if w not in stop_words][:3]
    
    return '_'.join(meaningful) if meaningful else 'report'


def _format_content(content: str, task: str, timestamp: datetime) -> str:
    """
    Format content with title and timestamp if needed.
    
    Args:
        content: Raw content to format
        task: Task description for title generation
        timestamp: When the report was created
        
    Returns:
        Formatted Markdown content
    """
    # If content already has a Markdown title, use as-is
    if content.strip().startswith('#'):
        return content
    
    # Add title and timestamp for plain text content
    formatted_title = f"# Report: {task}"
    timestamp_line = f"*Generated: {timestamp.strftime('%Y-%m-%d %H:%M')}*"
    
    return f"{formatted_title}\n\n{timestamp_line}\n\n{content}"


In [6]:
# assert os.environ["OPENAI_API_KEY"], "OPENAI_API_KEY is not set"
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"]
# )

# ---------- Agents ----------
supervisor = AssistantAgent(
    name="Supervisor",
    description="Oversees the process, approves final output, requests retries if needed.",
    model_client=model_client,
    system_message="You monitor progress and decide when to terminate the task.",
)

planner = AssistantAgent(
    name="Planner",
    model_client=model_client,
    system_message=(
        "You are a meticulous planner. Create a clear, step-by-step plan for the request."
        "Seek critical review from the Supervisor and critic."
        "Agree on the final plan."
        "End with PLAN_COMPLETE after reviews are successfully completed."
    ),
    description="Creates detailed step-by-step plans for tasks",
)

researcher = AssistantAgent(
    name="Researcher",
    description="Collects factual evidence using wiki_search, duckduckgo_search, produces report drafts.",
    model_client=model_client,
    tools=[wiki_search, duckduckgo_search],
    system_message="Research and produce a concise draft on the given topic using facts.",
)

critic = AssistantAgent(
    name="Critic",
    model_client=model_client,
    system_message=(
        """
You are a domain-agnostic Critic Agent. Your role: produce a concise, evidence-based critique of the provided artifact (plan, report, code, design, or prompt). Follow this exact process and style rules.

Process:
1. IDENTIFY: State artifact type, asserted goal, and audience (1 line).
2. SUMMARIZE: Give a 1–2 line neutral summary of content and intent.
3. STRENGTHS: List up to 5 concise strengths with line/section references.
4. ISSUES: For each issue, provide:
   - Title (one line)
   - Severity: {Critical, Major, Minor}
   - Evidence: exact quote / line numbers / code snippet
   - Impact: one sentence on consequences
   - Fix: Suggest corrective actions.
   - Tests to verify fix (if applicable)
5. PRIORITY LIST: Rank recommended fixes (highest impact first).
6. VERDICT: Single-line overall recommendation (Accept / Accept with changes / Major revision / Reject) and confidence level (High/Med/Low).
7. CLARIFY: If essential facts are missing for critical judgments, list the minimal questions required.
8. VERACITY: Assess the truthfulness and reliability of the information presented. Request sources or evidence for claims. Preferred format: [source](URL).
9. TRADEOFFS: Implore second-order consequences for unverified assumptions.

Style rules:
- Use active voice and action verbs.
- Keep each bullet ≤ 20 words.
- Use numbered lists and bullets.
- Cite exact lines or code references for claims.
- Challenge assumptions and seek clarification.
- End with a the author can act on.

Length limits:
- Summary: ≤ 40 words.
- Strengths: ≤ 5 bullets.
- Issues: ≤ 10 bullets.
- Overall critique: ≤ 500 words.

Tone: professional, concise, constructive.
"""
        "Do NOT APPROVE without addressing all concerns."
        "Respond with 'APPROVED: [...]' when your feedback is addressed."
    ),
    description="Reviews and provides constructive criticism; outputs 'APPROVED: [...]' when ready.",
)


editor = AssistantAgent(
    name="Editor",
    description="Formats approved drafts with proper citations, adapts structure to content type.",
    model_client=model_client,
    tools=[save_report],
    system_message=(
        """You are the Editor Agent - applying editorial rigor while adapting to content needs.

**Core Principles (Always Apply)**:
1. **Source Attribution**: Every factual claim needs a citation [^1], [^2], etc.
2. **Clear Structure**: Use appropriate headings for logical flow
3. **Professional Tone**: Clear, objective, and accessible language
4. **Evidence-Based**: Distinguish facts from analysis/opinion

**Flexible Formatting Guidelines**:

**For Analysis/Research Reports**:
- Use: Executive Summary, Key Findings, Analysis, Conclusions, References
- Structure: ## Main topics, ### Subtopics as needed

**For News/Current Events**:
- Use: Overview, Current Situation, Impact Analysis, References
- Lead with most important information first

**For Technical/Process Reports**:
- Use: Overview, Methodology, Results, Implementation, References
- Focus on actionable insights

**For Comparative Studies**:
- Use: Introduction, Comparison Framework, Analysis, Recommendations, References
- Tables and bullet points for comparisons

**Citation Rules (Flexible)**:
- Factual claims: "Power sector emissions increased 15%[^1]"
- Trends: "Studies indicate growing adoption[^2][^3]" 
- Analysis: "Based on available data[^1][^2], this suggests..."
- Web sources: Include URLs when available

**Reference Format (Adaptable)**:
```
## References
[^1]: Source Name. "Content/Query." Platform. [URL if available]
[^2]: Author/Organization. "Title." Retrieved from [Platform] search.
```

**Save Process**:
- Apply appropriate structure for content type
- Ensure all claims have citations
- Save using: save_report(content=formatted_content, task_description=original_task)
- Confirm save success and add 'TERMINATE'

**Quality Check**:
- Is the structure logical for this content type?
- Are sources properly attributed?
- Is the language clear and professional?
- Does it serve the reader's needs?

Adapt the structure to best serve the content and audience - not every report needs the same format."""
    ),
)

# ---------- Selector prompt with few-shot examples ----------
selector_prompt = """You are the selector agent. Choose exactly one agent name from {participants} to speak next based on the conversation history and their roles.

Few-shot examples:

Example 1:
History: Researcher produced a draft; Critic says 'Needs more sources and clarity'.
Selected agent: Researcher

Example 2:
History: Researcher produced a draft; Critic says 'APPROVED'.
Selected agent: Editor

Roles:
{roles}

Conversation history:
{history}

Rules:
- Pick the agent whose role most advances completion.

Return only the agent name (no extra text).
"""


# ---------- Termination callable ----------
max_messages = 30
txt_termination = TextMentionTermination("TERMINATE")
termination_condition = (
    MaxMessageTermination(max_messages=max_messages) | txt_termination
)

# ---------- Build SelectorGroupChat ----------
team = SelectorGroupChat(
    participants=[supervisor, planner, researcher, critic, editor],
    model_client=model_client,
    selector_prompt=selector_prompt,
    termination_condition=termination_condition,
)


# ---------- Selector validation ----------
def validate_selector_choice(raw_choice, participants):
    """Ensure selector returns a canonical agent name."""
    choice = raw_choice.strip().splitlines()[0].strip()
    for p in participants:
        if choice.lower() == p.name.lower():
            return p.name
    return None


In [7]:
import nest_asyncio
nest_asyncio.apply()


In [ ]:
@tenacity.retry(
    wait=tenacity.wait_exponential(multiplier=1, min=1, max=20),
    stop=tenacity.stop_after_attempt(2),
    retry=tenacity.retry_if_exception_type(Exception),
)
async def run_task(task_text: str):
    """
    Execute a multi-agent research task with proper termination handling.

    Args:
        task_text (str): The research task description

    Returns:
        str | None: Final report content or None if task incomplete
    """
    final_report = None
    task_completed = False

    try:
        logger.info(f"🚀 Starting task: {task_text}")

        # Let AutoGen handle termination internally - don't double-check
        async for event in team.run_stream(task=task_text):
            logger.info(f"📝 Event: {event}")

            # Check for TERMINATE keyword in message content
            if hasattr(event, "content") and isinstance(event.content, str):
                if "TERMINATE" in event.content:
                    final_report = event.content.split("TERMINATE", 1)[0].strip()
                    task_completed = True
                    logger.info("✅ Task completed with TERMINATE signal")
                    break

        # Save final state on completion
        await _save_team_state()

        if task_completed:
            logger.info("✅ Task completed successfully")
        else:
            logger.warning("⚠️ Task ended without explicit completion signal")

    except Exception as e:
        logger.error(f"❌ Exception during task execution: {e}")
        # Save state on failure for debugging
        await _save_team_state()
        raise

    return final_report


async def _save_team_state():
    """Helper function to save team state with error handling."""
    try:
        state = await team.save_state()
        with open("team_state.json", "w") as f:
            json.dump(state, f, indent=2)
        logger.info("💾 Team state saved successfully")
    except Exception as e:
        logger.error(f"❌ Failed to save team state: {e}")


async def resume_from_saved_state():
    """Resume execution from previously saved team state."""
    try:
        with open("team_state.json", "r") as f:
            saved_state = json.load(f)

        await team.load_state(saved_state)
        logger.info("🔄 Resuming from saved state")

        async for event in team.run_stream():
            logger.info(f"📝 Resume Event: {event}")

    except FileNotFoundError:
        logger.error("❌ No saved state file found")
    except Exception as e:
        logger.error(f"❌ Failed to resume from saved state: {e}")
        raise


if __name__ == "__main__":
    __TASK_QUERY = "Sectoral growth based on cyclics for 2025 and macro economic pressure and trade tariffs and uncertainty, which sectors are best poised for maximum investment returns in terms of % for the next year for a moderate to average risk profile investments"

    try:
        output = asyncio.run(run_task(__TASK_QUERY))
        if output:
            print("\n" + "=" * 60)
            print("🎯 FINAL REPORT")
            print("=" * 60)
            print(output)
        else:
            print("❌ Task did not complete successfully.")
    except Exception as e:
        logger.error(f"❌ Fatal error: {e}")
        print(f"❌ Execution failed: {e}")


2025-08-19 14:53:05.208 | INFO     | __main__:run_task:20 - 🚀 Starting task: Sectoral growth based on cyclics for 2025 and macro economic pressure and trade tariffs and uncertainty, which sectors are best poised for maximum investment returns in terms of % for the next year for a moderate to average risk profile investments
14:53:05:208 | INFO | __main__:20 | 🚀 Starting task: Sectoral growth based on cyclics for 2025 and macro economic pressure and trade tariffs and uncertainty, which sectors are best poised for maximum investment returns in terms of % for the next year for a moderate to average risk profile investments
2025-08-19 14:53:07.500 | INFO     | __main__:run_task:24 - 📝 Event: id='757b0ef3-2792-481b-bb87-e45d81d0a7bb' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 19, 12, 53, 5, 209338, tzinfo=datetime.timezone.utc) content='Sectoral growth based on cyclics for 2025 and macro economic pressure and trade tariffs and uncertainty, which sector